## Creating a logistic regression to predict CHD

### Import the relevant libraries

In [1]:
import pandas as pd
import numpy as np

### Load the data

In [2]:
data_preprocessed = pd.read_csv('CHD_preprocessed.csv')

In [3]:
data_preprocessed.head(10)

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,1,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,1,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,1,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0
5,0,43,0,0,0.0,0.0,0,1,0,228.0,180.0,110.0,30.30,77.0,99.0,0
6,0,63,0,0,0.0,0.0,0,0,0,205.0,138.0,71.0,33.11,60.0,85.0,1
7,0,45,0,1,20.0,0.0,0,0,0,313.0,100.0,71.0,21.68,79.0,78.0,0
8,1,52,0,0,0.0,0.0,0,1,0,260.0,141.5,89.0,26.36,76.0,79.0,0
9,1,43,0,1,30.0,0.0,0,1,0,225.0,162.0,107.0,23.61,93.0,88.0,0


In [4]:
data_preprocessed.isnull().sum()

male               0
age                0
education          0
currentSmoker      0
cigsPerDay         0
BPMeds             0
prevalentStroke    0
prevalentHyp       0
diabetes           0
totChol            0
sysBP              0
diaBP              0
BMI                0
heartRate          0
glucose            0
TenYearCHD         0
dtype: int64

### Creating the targets

In [5]:
targets = data_preprocessed['TenYearCHD']

In [6]:
## Checking the percentage of 1's and 0's
targets.sum() / targets.shape[0]

0.15194773772078393

### A comment on the targets

i will drop 'BMI', 'heartRate' and 'currentSmoker', because based on my summary table, they are not significant

In [7]:
data_with_targets = data_preprocessed.drop(['BMI', 'heartRate','currentSmoker'],axis=1)

### Select the inputs for the regression

In [8]:
data_with_targets.shape

(4133, 13)

In [9]:
unscaled_inputs = data_with_targets.iloc[:,:-1]

### Standardize the data

In [10]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler

class CustomScaler(BaseEstimator, TransformerMixin):
    
    def __init__(self,columns,copy=True,with_mean=True,with_std=True):
        self.scaler = StandardScaler(copy,with_mean,with_std)
        self.columns= columns
        self.mean_ = None
        self.var_ = None
        
    def fit(self, X,y=None):
        self.scaler.fit(X[self.columns],y)
        self.mean_ = np.mean(X[self.columns])
        self.var_ = np.var(X[self.columns])
        return self
    
    def transform(self, X,y=None, copy=None):
        init_col_order = X.columns
        X_scaled = pd.DataFrame(self.scaler.transform(X[self.columns]), columns=self.columns)
        X_not_scaled = X.loc[:,~X.columns.isin(self.columns)]
        return pd.concat([X_not_scaled, X_scaled],axis=1)[init_col_order]

In [11]:
unscaled_inputs.columns.values

array(['male', 'age', 'education', 'cigsPerDay', 'BPMeds',
       'prevalentStroke', 'prevalentHyp', 'diabetes', 'totChol', 'sysBP',
       'diaBP', 'glucose'], dtype=object)

In [12]:
# We will exclude the dummies (Male,education,currentSmoker,BPMeds,prevalentStroke,prevalentHyp,diabetes )
#columns_to_scale = ['age', 'cigsPerDay','totChol','sysBP', 'diaBP', 'BMI', 'heartRate', 'glucose']
        
columns_to_omit = ['male', 'education', 'currentSmoker',
       'BPMeds', 'prevalentStroke', 'prevalentHyp', 'diabetes','BMI', 'heartRate']

In [13]:
columns_to_scale = [x for x in unscaled_inputs.columns.values if x not in columns_to_omit]

In [39]:
chd_scaler = CustomScaler(columns_to_scale)

In [40]:
chd_scaler.fit(unscaled_inputs)

/Users/ozlemniksch/opt/anaconda3/lib/python3.7/site-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


CustomScaler(columns=['age', 'cigsPerDay', 'totChol', 'sysBP', 'diaBP',
                      'glucose'],
             copy=None, with_mean=None, with_std=None)

In [42]:
scaled_inputs = chd_scaler.transform(unscaled_inputs)

In [43]:
scaled_inputs

,male,age,education,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,glucose
0,1,-1.233235,1,-0.763751,0.0,0,0,0,-0.948992,-1.194286,-1.077067,-0.216401
1,0,-0.415535,0,-0.763751,0.0,0,0,0,0.303745,-0.514866,-0.156658,-0.260149
2,1,-0.181906,0,0.914524,0.0,0,0,0,0.189860,-0.220451,-0.240331,-0.522637
3,0,1.336681,1,1.753661,0.0,0,1,0,-0.265681,0.798679,1.014772,0.921047
4,0,-0.415535,1,1.166265,0.0,0,0,0,1.100941,-0.107215,0.094363,0.133583
...,...,...,...,...,...,...,...,...,...,...,...,...
4128,1,0.051723,0,-0.679837,0.0,0,1,0,1.738698,2.112224,0.763751,0.177331
4129,1,0.168537,1,2.844540,0.0,0,0,0,-0.675667,-0.265746,-0.240331,-0.610133
4130,0,-0.181906,0,0.914524,0.0,0,0,0,0.258191,-0.061920,-0.909720,0.177331
4131,0,-0.649163,0,0.494955,0.0,0,0,0,-0.607336,-0.265746,0.345384,0.002339


In [18]:
scaled_inputs.shape

(4133, 12)

### Split the data into train& test and shuffle

In [19]:
from sklearn.model_selection import train_test_split 

In [20]:
x_train,x_test,y_train,y_test = train_test_split(scaled_inputs, targets, test_size=0.2, random_state=2)

In [21]:
print( x_train.shape, y_train.shape)

(3306, 12) (3306,)


In [22]:
print( x_test.shape, y_test.shape)

(827, 12) (827,)


## Logistic resgression with sklearn

In [23]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [24]:
reg = LogisticRegression()

In [25]:
reg.fit(x_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [26]:
reg.score(x_train, y_train)

0.8578342407743497

### Calculating intercept and coefficient

In [27]:
reg.intercept_

array([-2.32918764])

In [28]:
reg.coef_

array([[ 0.48675167,  0.52624783,  0.09080897,  0.2431644 ,  0.1474719 ,
         0.91233759,  0.27631423,  0.18350065,  0.07517915,  0.35602062,
        -0.0762497 ,  0.14313439]])

In [29]:
## We will create a summary table, and need the column names
## scaled_inputs are np array, however unscaled_inputs are pd.dataframe

features = unscaled_inputs.columns.values
features

array(['male', 'age', 'education', 'cigsPerDay', 'BPMeds',
       'prevalentStroke', 'prevalentHyp', 'diabetes', 'totChol', 'sysBP',
       'diaBP', 'glucose'], dtype=object)

In [30]:
summary_table = pd.DataFrame(columns=['Feature Name'], data=features)
summary_table["Coefficient"] = np.transpose(reg.coef_)

summary_table

,Feature Name,Coefficient
0,male,0.486752
1,age,0.526248
2,education,0.090809
3,cigsPerDay,0.243164
4,BPMeds,0.147472
5,prevalentStroke,0.912338
6,prevalentHyp,0.276314
7,diabetes,0.183501
8,totChol,0.075179
9,sysBP,0.356021


In [31]:
## putting the intercept on the top of the table
summary_table.index = summary_table.index + 1

In [32]:
summary_table.loc[0] = ['Intercept', reg.intercept_[0]]

In [33]:
summary_table = summary_table.sort_index()
summary_table

,Feature Name,Coefficient
0,Intercept,-2.329188
1,male,0.486752
2,age,0.526248
3,education,0.090809
4,cigsPerDay,0.243164
5,BPMeds,0.147472
6,prevalentStroke,0.912338
7,prevalentHyp,0.276314
8,diabetes,0.183501
9,totChol,0.075179


when the coefficients (weights) are closer to 0, the smaller the weight

### Interpreting the coefficients

logistic regression formula:
- log(odds) = intercept + b1x1+b2x2+ ...+b14x14+b15x15
- that's why we are finding the exponentials 

In [34]:
summary_table['Odds_ratio'] = np.exp(summary_table.Coefficient)

In [35]:
summary_table

,Feature Name,Coefficient,Odds_ratio
0,Intercept,-2.329188,0.097375
1,male,0.486752,1.627023
2,age,0.526248,1.692570
3,education,0.090809,1.095060
4,cigsPerDay,0.243164,1.275278
5,BPMeds,0.147472,1.158901
6,prevalentStroke,0.912338,2.490137
7,prevalentHyp,0.276314,1.318262
8,diabetes,0.183501,1.201416
9,totChol,0.075179,1.078077


In [36]:
summary_table.sort_values('Odds_ratio', ascending=False)

,Feature Name,Coefficient,Odds_ratio
6,prevalentStroke,0.912338,2.490137
2,age,0.526248,1.692570
1,male,0.486752,1.627023
10,sysBP,0.356021,1.427637
7,prevalentHyp,0.276314,1.318262
4,cigsPerDay,0.243164,1.275278
8,diabetes,0.183501,1.201416
5,BPMeds,0.147472,1.158901
12,glucose,0.143134,1.153885
3,education,0.090809,1.095060


A feature is not particularly important:
- if its coefficient is around 0
- if its odds ratio is around 1

I decided to use backward elimanation based on odds ration on my summary table
    - BMI' s coef is 0.01 and odds ratio is 1.01 
    - currentSmoker's coef is -0.01 and odds ratio is 0.99
    - heartRate's coef is -0.02 and odds ratio is 0.98
These inputs are not bringing any or so much value to my model, so I will go back to target creation step and drop them. 

## Testing the model

In [37]:
reg.score(x_test,y_test)

0.848851269649335

Yey! My model is 85% accurate :D

## Saving the model

In [38]:
import pickle

In [44]:
with open('Framingham_CHD_model', 'wb') as file:
    pickle.dump(reg,file)

In [45]:
with open('scaler', 'wb') as file:
    pickle.dump(chd_scaler,file)